In [ ]:
% pylab inline
#% config InlineBackend.figure_format = 'svg'

import pandas as pd
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score

## Regularized Linear Regression Applied to the Ames Housing Data

Using the Ames Housing Data:

Dean De Cock
Truman State University
Journal of Statistics Education Volume 19, Number 3(2011), www.amstat.org/publications/jse/v19n3/decock.pdf



In this notebook, we will build some linear regression models to predict housing prices from this data.  We will split our data into training, validation, and test sets, build various (regularized) models on the training/validation data and compare their results on the test set. We will examine metrics such as *r-squared* and *mean absolute error*.

**Notebook Contents**

> 1. Simple EDA and dataset review
> 2. Exploring the behavior of LASSO vs. Ridge regularization
> 3. Standard-scaling features (a must for regularization!)
> 4. Tuning regularization strength via validation
> 5. Automated regularization strength tuning via cross-validation
> 6. _Bonus_: Using the LARS path to study feature importance 

## 1. Simple EDA and Dataset Review

#### Load the Data, Examine and Explore

In [ ]:
## Load in the Ames Housing Data
datafile = "data/Ames_Housing_Data.tsv"
df=pd.read_csv(datafile, sep='\t')

In [ ]:
## Examine the columns, look at missing data
df.info()

In [ ]:
# This is recommended by the data set author to remove a few outliers

df = df.loc[df['Gr Liv Area'] <= 4000,:]
df.shape

There are a *lot* of variables, many of which have a lot of missing values.  Let's pick out just a few columns and start building models using that.

In [ ]:
smaller_df= df.loc[:,['Lot Area', 'Overall Qual', 'Overall Cond', 
                      'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
                      'Full Bath', 'Bedroom AbvGr', 'Fireplaces', 
                      'Garage Cars','SalePrice']]

In [ ]:
smaller_df.describe()

In [ ]:
smaller_df.info()

In [ ]:
# There appears to be one NA in Garage Cars - fill with 0
smaller_df = smaller_df.fillna(0)

In [ ]:
smaller_df.info()

Now that we have a nice, filtered dataset, let's generate visuals to better understand the target and feature-target relationships: pairplot is great for this!

In [ ]:
sns.pairplot(smaller_df)

---
**Data Exploration Exercises**: 

1. What do these plots tell us about the distribution of the target?   

2. What do these plots tell us about the relationship between the features and the target? Do you think that linear regression is well-suited to this problem? Do any feature transformations come to mind?

3. What do these plots tell us about the relationship between various pairs of features? Do you think there may be any problems here? 

---

#### Setting up for modeling:

In [ ]:
#Separate our features from our target

X = smaller_df.loc[:,['Lot Area', 'Overall Qual', 'Overall Cond', 
                      'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
                      'Full Bath', 'Bedroom AbvGr', 'Fireplaces', 
                      'Garage Cars']]

y = smaller_df['SalePrice']

# create overall quality squared term, which we expect to 
# help based on the relationship we see in the pair plot 
X['OQ2'] = X['Overall Qual'] ** 2 

In [ ]:
X.info()

In [ ]:
#Split the data 60 - 20 - 20 train/val/test

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [ ]:
X_train.columns

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

## 2. Exploring the Behavior of LASSO vs. Ridge Regularization

We have been playing around with adding in or removing variables (or transformations of variables), and then seeing if the model improves.  However, this can be a tedious process.

Regularized Linear Regression tries to circumvent this by changing the *cost function*.  In "vanilla" linear regression, the coefficients are chosen purely to minimize the Sum of Squared Errors.  In Regularized Linear Regression, there is an additional component of the cost function that penalizes the "size" of the coefficients.  

Why penalize a coefficient?  At the simplest level, it forces a variable to be "worth it" in order to have a particularly extreme coefficient or even one that's greater than zero.  The intuition is that it is a "simpler model" to have smaller coefficients (in absolute value) than larger ones - regularization means that the coefficients are constrained to lie within a narrower region, making model coefficients more stable and less extreme.

Regularized Linear Regression introduces a **regularization strength parameter** that says how strongly we want to penalize the coefficients.  At one extreme there is no penalty, and we revert back to "vanilla" Linear Regression.  At the other extreme, the penalty is so onerous that we set all of the coefficients to zero.  In between these two extremes are a continuous set of models.  We will see how to choose the best value with *validation* or *cross-validation*.

There are two main "flavors" of Regularized Linear Regression.  In the **LASSO**, we penalize the sum of the absolute values of the coefficients and in **Ridge Regression** we penalize the sum of the squares of the coefficients. Which one works better depends on the data and the business needs of your model (e.g., strong inclination toward interpretability with small set of variables suggests use of LASSO). 

Let's see some examples.

In [ ]:
selected_columns = ['Lot Area', 'Overall Qual', 'Overall Cond', 
                    'Year Built','Year Remod/Add', 'Gr Liv Area', 
                    'Full Bath', 'Bedroom AbvGr', 'Fireplaces', 
                    'Garage Cars', 'OQ2']

In [ ]:
X_train.info()

In [ ]:
lasso_model = Lasso(alpha = 1000000) # this is a VERY HIGH regularization strength!, wouldn't usually be used
lasso_model.fit(X_train.loc[:,selected_columns], y_train)

In [ ]:
list(zip(selected_columns, lasso_model.coef_))

Notice how LASSO sets many variables to 0 (with a high enough alpha parameter). This is its ** feature selection ** property. To help remember that LASSO does this and not ridge, look no further than the name: **Least Absolute Shrinkage and Selection Operator** -- the shrinkage part means making the coefficients smaller by penalizing their size in the cost function, and the selection part is zeroing out coefficients. Remember the double SS! 

To highlight a notable difference between LASSO and ridge, we're going to add an additional column that's Lot Area + noise, so that we have two **highly collinear columns**. Then we'll fit a ridge and LASSO model and compare their coefficients.

In [ ]:
np.random.seed(6)

X_train_collinear = X_train.loc[:,selected_columns]
X_train_collinear['Lot Area Clone'] = (X_train_collinear['Lot Area'] + 
                                      2500 * np.random.randn(X_train.shape[0]))

X_train_collinear.corr() #notice .95 correlation b/w Lot Area and its "clone"

As a quick aside, let's understand what happens with p-values when there is a lot of collinearity! We are much less sure about our relationships being meaningful. In this case the model does detect the right variable as having a significant relationship, but this need not be the case in general.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf 

model = sm.OLS(y_train, sm.add_constant(X_train_collinear))
results = model.fit()

results.summary()

Now let's compare and contrast Ridge vs. Lasso

In [ ]:
lr_model_ridge = Ridge(alpha = 1000000000000)
lr_model_ridge.fit(X_train_collinear, y_train)

list(zip(X_train_collinear.columns, lr_model_ridge.coef_))

**Ridge** smoothed out all of the coefficients, **bringing them closer to 0 but not discarding any of them**. Also, it gave **roughly equal weight to the two highly collinear features**.

In [ ]:
lr_model_lasso = Lasso(alpha = 100000)
lr_model_lasso.fit(X_train_collinear, y_train)

list(zip(X_train_collinear.columns, lr_model_lasso.coef_))

Meanwhile, **Lasso zeroed out most of the coefficients**, and **dropped the noisy collinear clone**, performing feature selection to keep the features we really wanted. 

What are the Pros/Cons of either behavior?

**LASSO**:
* _Pro_: great for trimming features and focusing interpretation on a few key ones
* _Con_: risk of discarding features that are actually useful

**Ridge**:
* _Pro_: great for smoothly handling multicollinearity, very nice when working with sparse features 
* _Con_: will never fully discard features

As always, you have to validate to choose between the two. If the mapping from features to target truly depends on only a few key features, LASSO should outperform. If instead the target actually depends on many features (even if only a little dependent), Ridge should work better.  

In [ ]:
#Back to the original LASSO model: diagnostics

In [ ]:
test_set_pred = lasso_model.predict(X_test.loc[:,selected_columns])

In [ ]:
plt.scatter(test_set_pred, y_test, alpha=.1)
plt.plot(np.linspace(0,600000,1000), np.linspace(0,600000,1000))

In [ ]:
#r-squared
r2_score(y_test, test_set_pred)

In [ ]:
#Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true)) 

mae(y_test, test_set_pred)

## 3. Standard-scaling Features (a must for regularization!)

One issue with Regularized Linear Regression is that the "size" of a coefficient may be more reflective of the units or scale of the associated variable than the actual power of the relationship.  For example, if a distance is measured in millimeters it will have a larger coefficient than if it is measured in miles.  For this reason, best practice is to "standardize" the variables prior to running a regularized regression.  Standardizing means subtracting off each feature column's mean and then dividing by its standard deviation so that the resulting variable has mean 0 and standard deviation 1. This ensures that the variables are penalized fairly with respect to one another.

We demonstrate how to do this below.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
## This step fits the Standard Scaler to the training data
## Essentially it finds the mean and standard deviation of each variable in the training set

std = StandardScaler()
std.fit(X_train.values)

In [ ]:
## This step applies the scaler to the train set.
## It subtracts the mean it learned in the previous step and then divides by the standard deviation

X_tr = std.transform(X_train.values)

In [ ]:
## Apply the scaler to the test set

X_te = std.transform(X_test.values)

In [ ]:
## Note that even though we put a Pandas Dataframe into the scalar, what comes out is a numpy array
## In general, sklearn works on numpy.  It will accept pandas objects by trying to coerce them to numpy arrays
## But it will not usually output pandas objects

type(X_train), type(X_tr)

In [ ]:
## Here we can plot histograms of the transformed variables
## Note that they seem to have means of 0 and stddevs of 1
## (though they are not necessarily normally distributed)

plt.hist(X_tr[:,3])

Now that we have appropriately scaled our variables, we can apply the LASSO as before.

What we did before was technically not good practice since the variables were on different scales.  Certain variables would be (unfairly) penalized more than others.

In [ ]:
## Fit a LASSO model on the standardized data

lasso_model = Lasso(alpha = 10000)
lasso_model.fit(X_tr,y_train)

In [ ]:
## Note that now we can meaningful compare the importance of
## different features, since they're on the same scale

## But it's now difficult to interpret the coefficients
## We would need to translate back to the original feature scales by dividing
## each coefficient by the original column's standard deviation

list(zip(X_train.columns, lasso_model.coef_))

## 4. Tuning Regularization Strength via Validation

Here we will attempt to find the "best" value of the regularization strength alpha for this feature and target set and the LASSO model. We'll use simple validation (single train/valid split) as our model selection method.

We will first decide on a vector of "candidate" alpha values.  Then, for each candidate value, we run the following steps:

> 1. Fit a LASSO model on the training data
> 2. Using the newly trained model, make predictions on the validation data
> 3. Run evaluation metrics on validation

Then we plot how the errors change for the different values of alpha, and see where alpha minimizes our error metric on the validation data. This value of alpha is the one we would select for our final model.

In [ ]:
alphalist = 10**(np.linspace(-2,2,200))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))

for i,curr_alpha in enumerate(alphalist):

    # note the use of a new sklearn utility: Pipeline to pack
    # multiple modeling steps into one fitting process 
    steps = [('standardize', StandardScaler()), 
             ('lasso', Lasso(alpha = curr_alpha))]

    pipe = Pipeline(steps)
    pipe.fit(X_train.loc[:,selected_columns].values, y_train)
    
    val_set_pred = pipe.predict(X_val.loc[:,selected_columns].values)
    err_vec_val[i] = mae(y_val, val_set_pred)

In [ ]:
#plot the curve of validation error as alpha changes

plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
## This is the minimum error achieved on the validation set 
## across the different alpha values we tried

np.min(err_vec_val)

In [ ]:
## This is the value of alpha that gave us the lowest error
alphalist[np.argmin(err_vec_val)]

---
**Regularization Tuning Exercise**: 

Repeat the model selection workflow above (simple validation), but using a ridge model instead of a LASSO model. Based on the evidence you gather, do you think that a ridge or LASSO model has better predictive power on this dataset?

---

## 5. Automated Regularization Strength Tuning via Cross-validation 

### Using LassoCV to find the best alpha via Cross-Validation
In the previous section, we found the best alpha value by comparing the performance on a single validation set.  An even better, though more computationally intensive method, is to do a full cross-validation when comparing the different alphas. Fortunately, the `LassoCV` in sklearn handles this "under the hood". You pass the `LassoCV` the list of alphas and the number of folds to use for Cross-Validation.  

It will do the following for each candidate value of alpha:

> 1. Run cross-validation and score the result
> 2. Find the value of alpha that gave the best CV score
> 3. Fit a final model on all the data using the best value of alpha it just found

Then you can use the `predict` method of the model just as with all of our previous models

In [ ]:
## Scale the data as before
std = StandardScaler()
std.fit(X_train.values)

In [ ]:
## Scale the Predictors on both the train and test set
X_tr = std.transform(X_train.values)
X_te = std.transform(X_test.values)

In [ ]:
# Run the cross validation, find the best alpha, refit the model on all the data with that alpha

alphavec = 10**np.linspace(-2,2,200)

lasso_model = LassoCV(alphas = alphavec, cv=5)
lasso_model.fit(X_tr, y_train)

In [ ]:
# This is the best alpha value it found - not far from the value
# selected using simple validation
lasso_model.alpha_

In [ ]:
# These are the (standardized) coefficients found
# when it refit using that best alpha
list(zip(X_train.columns, lasso_model.coef_))

In [ ]:
# Make predictions on the test set using the new model
test_set_pred = lasso_model.predict(X_te)

In [ ]:
# Find the MAE and R^2 on the test set using this model
mae(y_test, test_set_pred)

In [ ]:
r2_score(y_test, test_set_pred)

---
**Regularization Tuning with CV Exercise**: 

Repeat the model selection workflow above (CV), but using the RidgeCV model instead of the LassoCV model. Based on the evidence you gather, do you think that a ridge or LASSO model has better predictive power on this dataset?

---

## 6. _Bonus_: Using the LARS Path to Study Feature Importance 

This is a tool used to visualize *all* of the LASSO models across a range of different alpha values.  At the far left is the value of alpha where the penalty on coefficients is *so* onerous, that it just sets all of the coefficients to zero.  At the far right is when there is no penalty, and corresponds to the values of the coefficients that you would get from a "vanilla" linear regression.

So each vertical slice corresponds to the coefficients you would get at a particular setting of alpha.  The black dotted lines indicate where a new variable "enters" the model (that is, its coefficient changes from 0 to non-zero).

This is a good way to see which variables are most influential and how their strengths change as you change the value of alpha. Intuitively, the features that enter the model (nonzero coefficients) earliest in the path are the ones that the model treats as most essential, that it doesn't want to live without.

In [ ]:
from sklearn.linear_model import lars_path

In [ ]:
## Scale the variables
std = StandardScaler()
std.fit(X_train.values)

In [ ]:
X_tr = std.transform(X_train.values)

In [ ]:
## Note: lars_path takes numpy matrices, not pandas dataframes

print("Computing regularization path using the LARS ...")
alphas, _, coefs = lars_path(X_tr, y_train.values, method='lasso')

In [ ]:
# plotting the LARS path

xx = np.sum(np.abs(coefs.T), axis=1)
xx /= xx[-1]

plt.figure(figsize=(10,10))
plt.plot(xx, coefs.T)
ymin, ymax = plt.ylim()
plt.vlines(xx, ymin, ymax, linestyle='dashed')
plt.xlabel('|coef| / max|coef|')
plt.ylabel('Coefficients')
plt.title('LASSO Path')
plt.axis('tight')
plt.legend(X_train.columns)
plt.show()